making psf bias DV and contaminate simulated theory datavector <br>
based on code from here:  https://github.com/des-science/Y3_shearcat_tests/blob/212c03bfc239c9cff5419efbe70bdfdba28639ec/alpha-beta-eta-test/code/produce_2pcfpsfbias.py
 <br> and here: https://github.com/des-science/Y3_shearcat_tests/blob/master/alpha-beta-eta-test/forecast/contaminate.py

In [1]:
import h5py
import getdist
from getdist import plots, MCSamples
from astropy.io import fits
import itertools

import numpy as np

# Make Contaminant

In [2]:
#the relevant files will all be in the TXdata folders:
# des: '../desy3/TXdata/'
# hsc: '../hscy3/TXdata/'
# kids: '../KiDS/TXdata/'

#copy and paste the path you want here:
path= '../desy3/TXdata/'

#DES and HSC have 4 bins, KiDS has 5.
nbins=5

In [3]:
#load rowes
with h5py.File(path+'rowe_stats.hdf5') as f:
    meanr = f['rowe_statistics/rowe_0_PSF-reserved/theta'][:]
    rho0p = f['rowe_statistics/rowe_0_PSF-reserved/xi_plus'][:]
    rho0m = f['rowe_statistics/rowe_0_PSF-reserved/xi_minus'][:]
    rho1p = f['rowe_statistics/rowe_1_PSF-reserved/xi_plus'][:]
    rho1m = f['rowe_statistics/rowe_1_PSF-reserved/xi_minus'][:]
    rho2p = f['rowe_statistics/rowe_2_PSF-reserved/xi_plus'][:]
    rho2m = f['rowe_statistics/rowe_2_PSF-reserved/xi_minus'][:]
    rho3p = f['rowe_statistics/rowe_3_PSF-reserved/xi_plus'][:]
    rho3m = f['rowe_statistics/rowe_3_PSF-reserved/xi_minus'][:]
    rho4p = f['rowe_statistics/rowe_4_PSF-reserved/xi_plus'][:]
    rho4m = f['rowe_statistics/rowe_4_PSF-reserved/xi_minus'][:]
    rho5p = f['rowe_statistics/rowe_5_PSF-reserved/xi_plus'][:]
    rho5m = f['rowe_statistics/rowe_5_PSF-reserved/xi_minus'][:]

In [4]:
#load tau fitting chains
alist = []
blist = []
elist = []
with h5py.File(path+'tau_chain.hdf5') as f:
    for n in range(nbins):
        alist.append(f[f'chain/bin_{n}/alpha'][:])
        blist.append(f[f'chain/bin_{n}/beta'][:])
        elist.append(f[f'chain/bin_{n}/eta'][:])

KeyError: 'Unable to open object (component not found)'

In [ ]:
a=[i for i in range(nbins)]
bin_pairs=[]
for p in itertools.combinations_with_replacement(a, 2): bin_pairs.append(p)
veclist = []
for i,j in bin_pairs:
    for z in range(len(alist[0])):
        dxip = [alist[i][z]*alist[i][z]*rho0p + blist[i][z]*blist[j][z]*rho1p + elist[i][z]*elist[j][z]*rho3p + (blist[i][z]*alist[j][z] + blist[j][z]*alist[i][z])*rho2p + (blist[i][z]*elist[j][z] + blist[j][z]*elist[i][z])*rho4p + (elist[i][z]*alist[j][z] +elist[j][z]*alist[i][z])*rho5p for i,j in bin_pairs]
        dxim = [alist[i][z]*alist[j][z]*rho0m + blist[i][z]*blist[j][z]*rho1m + elist[i][z]*elist[j][z]*rho3m + (blist[i][z]*alist[j][z] + blist[j][z]*alist[i][z])*rho2m + (blist[i][z]*elist[j][z] + blist[j][z]*elist[i][z])*rho4m + (elist[i][z]*alist[j][z] +elist[j][z]*alist[i][z])*rho5m for i,j in bin_pairs]
        dxi = dxip + dxim
        veclist.append(np.concatenate(np.c_[dxi]))
    covmat = np.cov(np.c_[veclist].T)



In [ ]:
al = []
bl = []
el = []
with h5py.File(path+'tau_stats.hdf5') as f:
    for n in range(nbins):
        al.append(np.asarray(f[f'tau_statistics/bestfits_PSF-reserved/bin_{n}/alpha']))
        bl.append(np.asarray(f[f'tau_statistics/bestfits_PSF-reserved/bin_{n}/beta']))
        el.append(np.asarray(f[f'tau_statistics/bestfits_PSF-reserved/bin_{n}/eta']))

In [ ]:
#Writting final contamination
hdu = fits.PrimaryHDU()
hdul = fits.HDUList([hdu])

dxip_list = []; dxim_list = []; bin1_list = []; bin2_list = []; angbin_list = []; ang_list = []
covxip_list = []; covxim_list = []


for i,j in bin_pairs:
    dxip = al[i]*al[j]*rho0p + bl[i]*bl[j]*rho1p + (bl[i]*al[j] + bl[j]*al[i])*rho2p + el[i]*el[j]*rho3p + (bl[i]*el[j] + bl[j]*el[i])*rho4p + (el[i]*al[j] +el[j]*al[i])*rho5p
    #(el[i]*al[j] +el[j]*al[i])*rho4p + (bl[i]*el[j] + bl[j]*el[i])*rho5p 
    dxim = al[i]*al[j]*rho0m + bl[i]*bl[j]*rho1m + (bl[i]*al[j] + bl[j]*al[i])*rho2m + el[i]*el[j]*rho3m + (bl[i]*el[j] + bl[j]*el[i])*rho4m + (el[i]*al[j] +el[j]*al[i])*rho5m
    #(el[i]*al[j] +el[j]*al[i])*rho4m + (bl[i]*el[j] + bl[j]*el[i])*rho5m


    ang_list.append(meanr)
    bin1_list.append(np.array( [i + 1]*len(meanr)))
    bin2_list.append(np.array( [j + 1]*len(meanr)))
    angbin_list.append(np.arange(len(meanr)))
    dxip_list.append(dxip)
    dxim_list.append(dxim)

hdul.insert(1, fits.ImageHDU(covmat, name='COVMAT'))
bin1array = np.concatenate(bin1_list)
bin2array = np.concatenate(bin2_list)
angbinarray = np.concatenate(angbin_list)
valuearray = np.concatenate(dxip_list)
angarray = np.concatenate(ang_list)

##Format of the fit file output
names=['BIN1', 'BIN2','ANGBIN', 'VALUE', 'ANG']
forms = ['i4', 'i4', 'i4',  'f4',  'f4']
dtype = dict(names = names, formats=forms)
nrows = len(angarray)
outdata = np.recarray((nrows, ), dtype=dtype)
array_list = [bin1array, bin2array, angbinarray, valuearray, angarray ]
for array, name in zip(array_list, names): outdata[name] = array 
corrhdu = fits.BinTableHDU(outdata, name='delta_xip')
hdul.insert(2, corrhdu)
valuearray = np.concatenate(dxim_list)
array_list = [bin1_array, bin2_array, angbin_array, valuearray, angarray ]
for array, name in zip(array_list, names): outdata[name] = array 
corrhdu = fits.BinTableHDU(outdata, name='delta_xim')
hdul.insert(3, corrhdu)
hdul.writeto('DES_contaminant.fits', overwrite=True)
print('Written!')

# Contaminate DV

In [2]:
#the theory data vector files will all be in the cosmosis output folders:
# des: '../cosmosis-files/datavec/theory/DESsimulated_theoryDV.fits'
# hsc: ''
# kids: '../cosmosis_files/datavec/theory/KiDSsimulated_theoryDV.fits'

#copy and paste the path you want here:
dv= '../cosmosis_files/datavec/theory/KiDSsimulated_theoryDV.fits'

#DES and HSC have 4 bins, KiDS has 5.
nbins=5

In [4]:
def get_error(covmatrix, lengths, name):
    import numpy as np
    if name is not None:
        if (name=='xip'):
            start = 0
            end =start + lengths[0]
        elif (name=='xim'):
            start = lengths[0]
            end =start + lengths[1]
        elif (name=='gammat'):
            start = lengths[0] + lengths[1]
            end =start + lengths[2]
        elif (name=='wtheta'):
            start = lengths[0] + lengths[1]+ lengths[2]
            end =start + lengths[3]
        return np.diagonal(covmatrix)[start:end]**0.5
    else:
        print("Correlation function not defined")
        return None

import fitsio
covmatrixfit_ori=fitsio.read(dv,ext=1)
xipfit_ori=fitsio.read(dv,ext=2)
ximfit_ori=fitsio.read(dv,ext=3)

#read contaminant
covmatrixfit_cont=fitsio.read('KiDS_contaminant-newbin.fits',ext=1)
xipfit_cont=fitsio.read('KiDS_contaminant-newbin.fits',ext=2)
ximfit_cont=fitsio.read('KiDS_contaminant-newbin.fits',ext=3)
dxipbin = xipfit_cont['VALUE']
dximbin = ximfit_cont['VALUE']

nsig = 2
lengths = [len(xipfit_cont), len(ximfit_cont)]
# run both, uncomment which one you need 
#upper bound:
#dxipbin += nsig*get_error(covmatrixfit_cont, lengths, 'xip')
#dximbin += nsig*get_error(covmatrixfit_cont, lengths, 'xim')
#lower bound:
dxipbin -= nsig*get_error(covmatrixfit_cont, lengths, 'xip')
dximbin -= nsig*get_error(covmatrixfit_cont, lengths, 'xim')

print("check: ", get_error(covmatrixfit_cont, lengths, 'xip'))
print(np.shape(dxipbin))
print(np.shape(dximbin))

a=[i for i in range(1,nbins+1)]
b=[j for j in range(1,nbins+1)]
bin_pairs=[]
for p in itertools.product(a, b):
    bin_pairs.append(p)
for i,j in bin_pairs:
    binp = (xipfit_ori['BIN1']==i)&(xipfit_ori['BIN2']==j)
    binm = (ximfit_ori['BIN1']==i)&(ximfit_ori['BIN2']==j)
    idxbinsp =  list(itertools.compress(range(len(binp)),  binp))
    idxbinsm =  list(itertools.compress(range(len(binm)),  binm))
    if (len(idxbinsp)!=0): xipfit_ori['VALUE'][binp] -=dxipbin[binp]
    if (len(idxbinsm)!=0): ximfit_ori['VALUE'][binm] -=dximbin[binm]

hdulist = fits.open(dv)
#delete all xip, xim but keep header
oldheaders =  [hdulist[2].header, hdulist[3].header]
hdulist.pop(index=2);
hdulist.pop(index=2);

xiphdu = fits.BinTableHDU(xipfit_ori)
ximhdu = fits.BinTableHDU(ximfit_ori)
hdulist.insert(2, xiphdu)
hdulist.insert(3, ximhdu)
hdulist[2].header = oldheaders[0]
hdulist[3].header = oldheaders[1]
hdulist.writeto('../cosmosis_files/datavec/contaminated/KiDS-1000_contaminated_2sig_inf.fits', overwrite=True)
print('written')


check:  [2.17119600e-06 1.35729726e-06 5.71545143e-07 2.79795815e-07
 2.08204443e-07 1.09307051e-07 4.60080523e-08 1.85153258e-08
 1.65836097e-08 1.70903659e-06 1.83042280e-06 1.95721113e-06
 1.74673302e-06 1.20150061e-06 6.40298617e-07 2.44743063e-07
 9.03422942e-08 8.19570054e-08 1.71683585e-06 1.65478802e-06
 1.57518332e-06 1.34729672e-06 9.28914941e-07 4.94495774e-07
 1.90437797e-07 7.08567822e-08 6.42266003e-08 1.54149614e-06
 1.57319859e-06 1.62982685e-06 1.44330538e-06 1.00100753e-06
 5.32332143e-07 2.04114275e-07 7.55028796e-08 6.83450059e-08
 2.05075064e-06 2.10115497e-06 2.11140826e-06 1.83245552e-06
 1.25263992e-06 6.67202456e-07 2.55387887e-07 9.47151634e-08
 8.59867968e-08 1.37160732e-06 8.33505452e-07 2.94252817e-07
 8.60391458e-08 4.54449780e-08 2.36830560e-08 1.11352886e-08
 5.54344400e-09 5.13735936e-09 8.62395938e-07 7.84937393e-07
 7.29146934e-07 6.20634024e-07 4.23675280e-07 2.25542413e-07
 8.64114809e-08 3.21151349e-08 2.91586285e-08 1.23358887e-06
 9.87787977e-07 